# آموزش RL برای متعادل کردن Cartpole

این نوت‌بوک بخشی از [برنامه درسی AI برای مبتدیان](http://aka.ms/ai-beginners) است. این آموزش از [آموزش رسمی PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) و [این پیاده‌سازی Cartpole با PyTorch](https://github.com/yc930401/Actor-Critic-pytorch) الهام گرفته شده است.

در این مثال، از یادگیری تقویتی (RL) برای آموزش مدلی استفاده خواهیم کرد که بتواند یک میله را روی یک چرخ‌دستی که می‌تواند به چپ و راست روی یک محور افقی حرکت کند، متعادل نگه دارد. برای شبیه‌سازی این میله از محیط [OpenAI Gym](https://www.gymlibrary.ml/) استفاده خواهیم کرد.

> **توجه**: می‌توانید کد این درس را به صورت محلی (مثلاً از طریق Visual Studio Code) اجرا کنید، که در این صورت شبیه‌سازی در یک پنجره جدید باز خواهد شد. اگر کد را به صورت آنلاین اجرا می‌کنید، ممکن است نیاز باشد برخی تغییرات در کد اعمال کنید، همان‌طور که [اینجا](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) توضیح داده شده است.

ابتدا مطمئن می‌شویم که Gym نصب شده است:


In [ ]:
import sys
!{sys.executable} -m pip install gym

حالا بیایید محیط CartPole را ایجاد کنیم و ببینیم چگونه می‌توانیم با آن کار کنیم. یک محیط دارای ویژگی‌های زیر است:

* **فضای اقدام** مجموعه‌ای از اقدامات ممکن است که می‌توانیم در هر مرحله از شبیه‌سازی انجام دهیم  
* **فضای مشاهده** فضایی است که مشاهداتی که می‌توانیم انجام دهیم در آن قرار دارد  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

بیایید ببینیم شبیه‌سازی چگونه کار می‌کند. حلقه زیر شبیه‌سازی را اجرا می‌کند تا زمانی که `env.step` پرچم پایان `done` را برنگرداند. ما به صورت تصادفی اقدامات را با استفاده از `env.action_space.sample()` انتخاب می‌کنیم، که به این معناست که آزمایش احتمالاً خیلی سریع شکست خواهد خورد (محیط CartPole زمانی پایان می‌یابد که سرعت CartPole، موقعیت یا زاویه آن خارج از محدودیت‌های مشخص باشد).

> شبیه‌سازی در یک پنجره جدید باز خواهد شد. شما می‌توانید کد را چندین بار اجرا کنید و ببینید چگونه رفتار می‌کند.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

شما می‌توانید مشاهده کنید که مشاهدات شامل ۴ عدد هستند. این اعداد عبارتند از:
- موقعیت چرخ‌دستی
- سرعت چرخ‌دستی
- زاویه میله
- نرخ چرخش میله

`rew` پاداشی است که در هر مرحله دریافت می‌کنیم. در محیط CartPole، شما برای هر مرحله شبیه‌سازی ۱ امتیاز دریافت می‌کنید و هدف این است که مجموع پاداش‌ها را به حداکثر برسانید، یعنی مدت زمانی که CartPole می‌تواند بدون افتادن تعادل خود را حفظ کند.

در یادگیری تقویتی، هدف ما آموزش یک **سیاست** $\pi$ است که برای هر حالت $s$ به ما بگوید کدام عمل $a$ را انجام دهیم، به عبارت دیگر $a = \pi(s)$.

اگر بخواهید یک راه‌حل احتمالی داشته باشید، می‌توانید سیاست را به‌عنوان بازگرداندن مجموعه‌ای از احتمالات برای هر عمل تصور کنید، یعنی $\pi(a|s)$ به معنای احتمال انجام عمل $a$ در حالت $s$ خواهد بود.

## روش گرادیان سیاست

در ساده‌ترین الگوریتم RL، که **گرادیان سیاست** نامیده می‌شود، ما یک شبکه عصبی را آموزش می‌دهیم تا عمل بعدی را پیش‌بینی کند.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

ما شبکه را با اجرای آزمایش‌های متعدد آموزش خواهیم داد و پس از هر اجرا شبکه خود را به‌روزرسانی می‌کنیم. بیایید تابعی تعریف کنیم که آزمایش را اجرا کرده و نتایج (به‌اصطلاح **ردپا**) - تمام حالت‌ها، اقدامات (و احتمالات پیشنهادی آن‌ها) و پاداش‌ها - را بازگرداند:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

شما می‌توانید یک قسمت را با شبکه آموزش‌ندیده اجرا کنید و مشاهده کنید که پاداش کل (یعنی طول قسمت) بسیار کم است:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

یکی از جنبه‌های پیچیده الگوریتم گرادیان سیاست، استفاده از **پاداش‌های تخفیف‌خورده** است. ایده این است که بردار مجموع پاداش‌ها را در هر مرحله از بازی محاسبه کنیم و در این فرآیند، پاداش‌های اولیه را با استفاده از ضریب $gamma$ تخفیف دهیم. همچنین بردار حاصل را نرمال‌سازی می‌کنیم، زیرا قصد داریم از آن به عنوان وزن برای تأثیرگذاری بر آموزش خود استفاده کنیم:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

حالا بیایید آموزش واقعی را شروع کنیم! ما ۳۰۰ قسمت اجرا خواهیم کرد و در هر قسمت مراحل زیر را انجام می‌دهیم:

1. آزمایش را اجرا کرده و ردگیری (trace) را جمع‌آوری می‌کنیم.
2. تفاوت (`gradients`) بین اقدامات انجام‌شده و احتمالات پیش‌بینی‌شده را محاسبه می‌کنیم. هرچه این تفاوت کمتر باشد، بیشتر مطمئن می‌شویم که اقدام درست را انجام داده‌ایم.
3. پاداش‌های تخفیف‌خورده را محاسبه کرده و گرادیان‌ها را در پاداش‌های تخفیف‌خورده ضرب می‌کنیم - این کار تضمین می‌کند که گام‌هایی با پاداش‌های بالاتر تأثیر بیشتری بر نتیجه نهایی داشته باشند نسبت به گام‌هایی با پاداش‌های کمتر.
4. اقدامات هدف مورد انتظار برای شبکه عصبی ما تا حدی از احتمالات پیش‌بینی‌شده در طول اجرا گرفته می‌شود و تا حدی از گرادیان‌های محاسبه‌شده. ما از پارامتر `alpha` برای تعیین میزان تأثیر گرادیان‌ها و پاداش‌ها استفاده می‌کنیم - این مفهوم به عنوان *نرخ یادگیری* الگوریتم تقویتی شناخته می‌شود.
5. در نهایت، شبکه خود را بر اساس حالت‌ها و اقدامات مورد انتظار آموزش می‌دهیم و این فرآیند را تکرار می‌کنیم.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

حالا بیایید قسمت را با رندر اجرا کنیم تا نتیجه را ببینیم:


In [ ]:
_ = run_episode(render=True)

امیدوارم متوجه شده باشید که اکنون میله می‌تواند به‌خوبی تعادل خود را حفظ کند!

## مدل Actor-Critic

مدل Actor-Critic توسعه‌ای پیشرفته‌تر از گرادیان‌های سیاستی (policy gradients) است که در آن یک شبکه عصبی طراحی می‌کنیم تا هم سیاست (policy) و هم پاداش‌های تخمینی را یاد بگیرد. این شبکه دو خروجی خواهد داشت (یا می‌توانید آن را به‌عنوان دو شبکه جداگانه در نظر بگیرید):
* **Actor** عملی را که باید انجام شود پیشنهاد می‌دهد و توزیع احتمالات حالت را به ما می‌دهد، مشابه مدل گرادیان سیاستی.
* **Critic** تخمین می‌زند که پاداش حاصل از آن اقدامات چه خواهد بود. این بخش مجموع پاداش‌های تخمینی در آینده را در حالت فعلی بازمی‌گرداند.

بیایید چنین مدلی را تعریف کنیم:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

ما نیاز داریم که توابع `discounted_rewards` و `run_episode` خود را کمی تغییر دهیم:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


اکنون حلقه اصلی آموزش را اجرا خواهیم کرد. ما از فرآیند آموزش دستی شبکه با محاسبه توابع زیان مناسب و به‌روزرسانی پارامترهای شبکه استفاده خواهیم کرد:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## نکات کلیدی

ما در این نمایش دو الگوریتم یادگیری تقویتی را مشاهده کردیم: گرادیان سیاست ساده و بازیگر-منتقد پیچیده‌تر. می‌توانید ببینید که این الگوریتم‌ها با مفاهیم انتزاعی حالت، عمل و پاداش کار می‌کنند - بنابراین می‌توان آن‌ها را در محیط‌های بسیار متفاوت اعمال کرد.

یادگیری تقویتی به ما این امکان را می‌دهد که بهترین استراتژی برای حل مسئله را تنها با نگاه کردن به پاداش نهایی یاد بگیریم. این واقعیت که نیازی به مجموعه داده‌های برچسب‌گذاری‌شده نداریم، به ما اجازه می‌دهد شبیه‌سازی‌ها را بارها تکرار کنیم تا مدل‌های خود را بهینه کنیم. با این حال، هنوز چالش‌های زیادی در یادگیری تقویتی وجود دارد که اگر تصمیم بگیرید بیشتر روی این حوزه جذاب از هوش مصنوعی تمرکز کنید، ممکن است آن‌ها را بیاموزید.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
